#### Replicating [Chan et. al 2023](https://ieeexplore.ieee.org/document/10156650/) Fig. 3
Differences: 
- nominal
- no iterations
- not robust
- 1 scenario only

In [34]:
# ex. cl_data
s = np.load('data/trial_exp/trial0_iter0_sim_data.npy', allow_pickle=True).item()
sim_data = s['rep0']
sim_data = sim_data['sim_data']


In [44]:
np.shape(sim_data['Usim'])

(2, 240)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from casadi import *
from copy import *
import scipy.io

### Loading in information

In [2]:
model = scipy.io.loadmat('data/mat/APPJmodel_2022_09_22_17h28m06s.mat') # load in model data

In [3]:
ts = 0.5 # sampling time
Tref = 43.0 # reference temperature
Tmax = 45.0 # max temp. for patient comfort
Np = 5 # prediction horizon

Kcem = 0.5

In [4]:
# Linear model used for control
A, B = model['A'], model['B']
C, D = model['C'], 0 # C = identity, D = 0

xss = np.ravel(model['yss']) 
uss = np.ravel(model['uss'])

In [5]:
x0 = np.zeros((2,))

In [6]:
nx, nu, ny = A.shape[1], B.shape[1], C.shape[0] # num. states, inputs (q, P), outputs (Ts, I)
nyc = 1 # num. controlled outputs

In [7]:
# constraint bounds
u_min, u_max = np.array([1.5, 1.5]) - uss, np.array([5, 5]) - uss
x_min, x_max = np.array([25,0]) - xss, np.array([Tmax, 80]) - xss
y_min, y_max = x_min, x_max

u_init, x_init, y_init = (u_min + u_max) / 2, (x_min + x_max) / 2, (y_min + y_max) / 2

In [8]:
# casadi functions
x, u, yref, Ts = SX.sym('x', nx), SX.sym('u', nu), SX.sym('yref', nyc), SX.sym('Ts', nyc)

In [9]:
# dynamics functions (prediction model)
xnext = A@x + B@u
f = Function('f', [x, u], [xnext])

# output equation (control model)
y = C@x
h = Function('h', [x], [y])

In [10]:
# CEM output--> K is an exponential based (substracte), Tref = 43'C, ts = sampling time
CEM = Kcem ** (Tref - Ts) * ts / 60 

# stage cost (nonlinear CEM cost)
lstg = Kcem ** (Tref - (x[0] + xss[0])) * ts / 60
lstage = Function('lstage', [x], [lstg])

### Set up solver

In [11]:
# containers
X = [0 for _ in range(Np + 1)]
U = [0 for _ in range(Np)]
Y = [0 for _ in range(Np + 1)]

In [12]:
J = 0 # init. cost/objective function

In [13]:
opti = Opti()

In [14]:
CEMref = opti.parameter(nyc) # target/reference output, fixed during optimization
opti.set_value(CEMref, np.zeros((nyc, 1))) # set value of parameter

CEM0 = opti.parameter(nyc) # initial CEM 
opti.set_value(CEM0, np.zeros((nyc, 1))) # set value of initial CEM parameter

X[0] = opti.parameter(nx) # initial state as parameter
opti.set_value(X[0], np.zeros((nx, 1))) # set value of initial state parameter

Y[0] = opti.variable(ny) # output as decision variable 
opti.subject_to(Y[0] == h(X[0])) # define output constraints based on initial state
opti.set_initial(Y[0], y_init) # define initial output 

In [15]:
for k in range(Np): 
    # controller @ k
    U[k] = opti.variable(nu) # controller input as decision variable
    opti.subject_to(opti.bounded(u_min, U[k], u_max)) # define controller input constraints
    opti.set_initial(U[k], u_init) # define initial controller input

    # increment stage cost 
    Jstage = lstage(X[k])
    J += Jstage

    # state @ k+1
    X[k + 1] = opti.variable(nx) # state at k+1 as decision variable
    opti.subject_to(opti.bounded(x_min, X[k + 1], x_max)) # define state constraints
    opti.set_initial(X[k + 1], x_init) # define initial state 

    # output @ k+1
    Y[k + 1] = opti.variable(ny) # output at k+1 as decision variable
    opti.subject_to(opti.bounded(y_min, Y[k +1], y_max)) # define state constraints
    opti.set_initial(Y[k + 1], y_init) # define initial output

    # dynamics constraint
    opti.subject_to(X[k + 1] == f(X[k], U[k]))
    # output equality constraint
    opti.subject_to(Y[k + 1] == h(X[k + 1]))

In [16]:
# calculate the terminal cost, note no splits or iterations
J_end = lstage(X[-1])
Jcon = J + CEM0

J = (Jcon - CEMref) ** 2 # eq. 6 control objective 

# minimize cost
opti.minimize(J)

In [17]:
# solver info
p_opts = {'verbose': False, 'expand': True, 'print_time': 1} # options taken from K.C. --> Ipopt options to print to console
s_opts = {'max_iter': 1000, 'print_level': 1, 'tol': 1e-6}

opti.solver('ipopt', p_opts, s_opts)

soln = opti.solve()


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 119.00us ( 10.82us)  48.31us (  4.39us)        11
       nlp_g  | 186.00us ( 16.91us)  70.09us (  6.37us)        11
  nlp_grad_f  | 177.00us ( 14.75us)  73.45us (  6.12us)        12
  nlp_hess_l  | 142.00us ( 14.20us)  61.47us (  6.15us)        10
   nlp_jac_g  | 192.00us ( 16.00us)  77.90us (  6.49us)        12
       total  |  43.33ms ( 43.33ms)  18.07ms ( 18.07ms)         1


In [18]:
# list of containers 

# prediction values
U_soln = np.asarray([soln.value(u) for u in U]).reshape(-1, nu).T
X_soln = np.asarray([soln.value(x) for x in X]).reshape(-1, nx).T
Y_soln = np.asarray([soln.value(y) for y in Y]).reshape(-1, ny).T

U_sim = np.zeros((nu, Np))
X_sim = np.zeros((nx, Np + 1))
Y_sim = np.zeros((ny, Np + 1))

X_sim[:, 0] = np.ravel(x0) # set initial state
Y_sim[:, 0] = np.ravel(h(X_sim[:, 0]).full())

In [19]:
# run closed-loop solution, nominal e.g. no noise
for k in range(Np):
    opti.set_value(X[0], X_sim[:, k]) # set initial state parameters 
    sol = opti.solve() # solve optimization problem
    U_soln = np.asarray([sol.value(u) for u in U]).reshape(nu, -1) # extract control inputs based on state
    Y_soln = np.asarray([sol.value(y) for y in Y]).reshape(ny, -1) # extract outputs based on state

    U_sim[:, k] = U_soln[:, 0] # for control, only interested in the first opt. control input
    X_sim[:, k + 1] = np.ravel(f(X_sim[:, k], U_sim[:, k]).full()) # apply input to state system/plant
    Y_sim[:, k + 1] = np.ravel(h(X_sim[:, k + 1]).full()) # apply input to output system/plant

      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 205.00us (  9.32us)  89.69us (  4.08us)        22
       nlp_g  | 289.00us ( 13.14us) 112.09us (  5.09us)        22
    nlp_grad  |  19.00us ( 19.00us)   9.85us (  9.85us)         1
  nlp_grad_f  | 258.00us ( 10.75us) 112.49us (  4.69us)        24
  nlp_hess_l  | 212.00us ( 10.60us)  95.48us (  4.77us)        20
   nlp_jac_g  | 300.00us ( 12.50us) 117.26us (  4.89us)        24
       total  |  17.46ms ( 17.46ms)   8.72ms (  8.72ms)         1
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 233.00us (  7.06us) 114.42us (  3.47us)        33
       nlp_g  | 338.00us ( 10.24us) 148.21us (  4.49us)        33
    nlp_grad  |  28.00us ( 14.00us)  14.15us (  7.08us)         2
  nlp_grad_f  | 297.00us (  8.25us) 142.35us (  3.95us)        36
  nlp_hess_l  | 239.00us (  7.97us) 121.54us (  4.05us)        30
   nlp_jac_g  | 334.00us (  9.28us) 149.31us (  4.15us)        36
       tot

### Plotting

State and input profiles of closed-loop experiments for a single iteration 

States  X (CEM, Ts)

Control U (P  , q )

In [21]:
X_sim

array([[ 0.        , -0.5267362 , -1.23786599, -1.89058978, -2.46939859,
        -2.98009639],
       [ 0.        , -4.8854003 , -5.42593023, -5.52714878, -5.57183955,
        -5.60019779]])